# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install -r "/content/drive/MyDrive/LLM/requirements.txt"

In [ ]:
import os  ## step-3
# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLM/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set
!echo $OPENAI_API_KEY

In [ ]:
import openai
client = openai.OpenAI() #step-4

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Generate a marketing product description from a product fact sheet

In [ ]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our stylish and versatile mid-century inspired office chair, perfect for both home and business settings. This chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more.

Customize your chair with several options of shell color and base finishes to match your decor. Choose between plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The chair is also available with or without armrests to suit your preference.

Constructed with a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy raise/lower action, this chair is both durable and functional. The dimensions of the chair are as follows: WIDTH 53 CM | 20.87”, DEPTH 51 CM | 20.08”, HEIGHT 80 CM | 31.50”, SEAT HEIGHT 44 CM | 17.32”, SEAT DEPTH 41 CM | 16.14”.

Choose between soft or hard-floor caster options and two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). You

## Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Available in various colors and finishes, with options for upholstery and armrests. Designed for both home and business use, with a durable construction and adjustable height feature. Made in Italy for quality assurance.


In [ ]:
len(response.split())

47

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile and stylish office chair, part of a mid-century inspired furniture collection. Constructed with a durable cast aluminum shell and base glider coated with modified nylon. The seat features high-density foam for comfort. Available in various colors and finishes, perfect for any home or business setting. Made in Italy.


In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our versatile mid-century office chair, featuring customizable shell colors and base finishes. Choose from plastic or full upholstery in various fabrics and leathers. With a durable aluminum base and pneumatic adjust, this chair is perfect for any setting. Product IDs: SWC-100, SWC-110. Made in Italy.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<p>This mid-century inspired office chair is a perfect addition to any home or business setting. With a variety of shell colors and base finishes to choose from, you can customize this chair to fit your style. The chair is available with plastic back and front upholstery or full upholstery in a range of fabric and leather options. The 5-wheel plastic coated aluminum base provides stability, while the pneumatic chair adjust allows for easy raise/lower action. Whether you prefer armless or armrests, this chair is designed for comfort and functionality. Made with high-quality materials, including cast aluminum with modified nylon coating and HD36 foam, this chair is built to last. Bring a touch of Italian craftsmanship into your space with this stylish and versatile office chair.</p>

<p>Product IDs: SWC-100, SWC-110</p>
</div>

<table>
  <caption>Product Dimensions</caption>
  <tr>
    <th>Dimension</th>
    <th>Measurements</th>
  </tr>
  <tr>
    <td>WIDTH</td>
    <td>20.87"</td

## Load Python libraries to view HTML

In [ ]:
from IPython.display import display, HTML

In [ ]:
display(HTML(response))

Dimension,Measurements
WIDTH,"20.87"""
DEPTH,"20.08"""
HEIGHT,"31.50"""
SEAT HEIGHT,"17.32"""
SEAT DEPTH,"16.14"""


## Try experimenting on your own!